## Preprocessing

In [46]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import pandas as pd
import tensorflow as tf
from pprint import pprint
import keras_tuner as kt

%run helper_functions.ipynb

# Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


__Target Variable__: `IS_SUCCESSFUL`

__Feature Variables__: All other columns

In [47]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [48]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [49]:
# Limit columns to 10 unique values
limit_unique(
    application_df,
    max_value = 10,
    columns_to_limit = ['APPLICATION_TYPE', 'CLASSIFICATION']
)

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64
Number of unique values: 10

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other      887
C7000      777
C1700      287
C4000      194
C5000      116
Name: count, dtype: int64
Number of unique values: 10



### Map to Binary Integers

In [50]:
# Since binary, convert SPECIAL_CONSIDERATIONS to 0s and 1s.
print(f"Before conversion: {application_df['SPECIAL_CONSIDERATIONS'].value_counts()}\n")

application_df['SPECIAL_CONSIDERATIONS'] = application_df['SPECIAL_CONSIDERATIONS'].map({
    'N': 0,
    'Y': 1
})
print(f"After conversion: {application_df['SPECIAL_CONSIDERATIONS'].value_counts()}")

Before conversion: SPECIAL_CONSIDERATIONS
N    34272
Y       27
Name: count, dtype: int64

After conversion: SPECIAL_CONSIDERATIONS
0    34272
1       27
Name: count, dtype: int64


### Ordinal Encoding
- There is no data on: `500000-1M`
- Solution: Convert to ordinal encoding to account for this

In [51]:
# Convert each value to an ordinal score
income_map = {
    '0': 0,
    '1-9999': 1,
    '10000-24999': 2,
    '25000-99999': 3,
    '100000-499999': 4,
    '500000-1M': 5, # No data in this category
    '1M-5M': 6,
    '5M-10M': 7,
    '10M-50M': 8,
    '50M+': 9
}
application_df['ORDINAL_INCOME_AMT'] = application_df['INCOME_AMT'].map(income_map)

### Numerical Income Amounts

In [52]:
# Display all possible values for INCOME_AMT
application_df['INCOME_AMT'].value_counts()

INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: count, dtype: int64

In [53]:
# Split the bounded values by the hyphen and create new columns
application_df[['LOWER_INCOME', 'UPPER_INCOME']] = application_df['INCOME_AMT'].str.split('-', expand=True)

In [54]:
# If INCOME_AMT contains "M", convert to e6.
application_df['UPPER_INCOME'] = application_df['UPPER_INCOME'].str.replace('M', 'e6')
application_df['LOWER_INCOME'] = application_df['LOWER_INCOME'].str.replace('M', 'e6')

# Confirm changes
display(application_df[['LOWER_INCOME', 'UPPER_INCOME']].value_counts())
application_df.head()

LOWER_INCOME  UPPER_INCOME
25000         99999           3747
100000        499999          3374
1e6           5e6              955
1             9999             728
10000         24999            543
10e6          50e6             240
5e6           10e6             185
Name: count, dtype: int64

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,ORDINAL_INCOME_AMT,LOWER_INCOME,UPPER_INCOME
0,T10,Independent,C1000,ProductDev,Association,1,0,0,5000,1,0,0,None
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,0,108590,1,1,1,9999
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,0,5000,0,0,0,None
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,0,6692,1,2,10000,24999
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,0,142590,1,4,100000,499999


In [55]:
# Address 'None' values in UPPER_INCOME. If INCOME_AMT is '0', set UPPER_INCOME to 0.
application_df.loc[application_df['LOWER_INCOME'] == '0', 'UPPER_INCOME'] = 0
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,ORDINAL_INCOME_AMT,LOWER_INCOME,UPPER_INCOME
0,T10,Independent,C1000,ProductDev,Association,1,0,0,5000,1,0,0,0
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,0,108590,1,1,1,9999
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,0,5000,0,0,0,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,0,6692,1,2,10000,24999
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,0,142590,1,4,100000,499999


In [56]:
# Address INCOME_AMT values with `50M+`
application_df.loc[application_df['INCOME_AMT'] == '50M+']

# Convert UPPER_INCOME NaN values to 50M
application_df.loc[application_df['INCOME_AMT'] == '50M+', 'UPPER_INCOME'] = 50e6

# Convert to '50M+' to 50e6
application_df['LOWER_INCOME'] = application_df['LOWER_INCOME'].replace('50e6+', '50e6')

# Convert column to integer
application_df['LOWER_INCOME'] = pd.to_numeric(application_df['LOWER_INCOME']).astype(int)
application_df['UPPER_INCOME'] = pd.to_numeric(application_df['UPPER_INCOME']).astype(int)

In [57]:
# Create an average column
application_df['AVG_INCOME'] = application_df[['LOWER_INCOME', 'UPPER_INCOME']].mean(axis=1)

### Ask Amount vs Average Income Comparison

In [58]:
# Create a new column that compares the ASK_AMT to LOWER_INCOME
application_df['ASK_VS_INCOME'] = application_df['ASK_AMT'] > application_df['AVG_INCOME']

# Convert boolean to Y/N for consistency
application_df['ASK_VS_INCOME'] = application_df['ASK_VS_INCOME'].map({
    False: 0,
    True: 1
})

# Display updated dataframe
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,ORDINAL_INCOME_AMT,LOWER_INCOME,UPPER_INCOME,AVG_INCOME,ASK_VS_INCOME
0,T10,Independent,C1000,ProductDev,Association,1,0,0,5000,1,0,0,0,0.0,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,0,108590,1,1,1,9999,5000.0,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,0,5000,0,0,0,0,0.0,1
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,0,6692,1,2,10000,24999,17499.5,0
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,0,142590,1,4,100000,499999,299999.5,0


### Combination Columns

In [59]:
# Create a new combination column
application_df['AFFILIATION_ORGANIZATION'] = application_df['AFFILIATION'] + "_" + application_df['ORGANIZATION']
application_df['AFFILIATION_ORGANIZATION'].value_counts()

AFFILIATION_ORGANIZATION
Independent_Trust                16212
CompanySponsored_Association      8479
CompanySponsored_Trust            7206
Independent_Association           1764
Independent_Co-operative           465
Family/Parent_Trust                 55
Independent_Corporation             39
National_Trust                      29
CompanySponsored_Co-operative       18
Regional_Trust                      11
Family/Parent_Association            9
National_Association                 2
National_Co-operative                2
Other_Corporation                    2
CompanySponsored_Corporation         2
Other_Trust                          2
Regional_Association                 1
Regional_Co-operative                1
Name: count, dtype: int64

In [60]:
# Create another combination column
application_df['AFFILIATION_USECASE'] = application_df['AFFILIATION'] + "_" + application_df['USE_CASE']
application_df['AFFILIATION_USECASE'].value_counts()

AFFILIATION_USECASE
Independent_Preservation          16218
CompanySponsored_Preservation     11779
CompanySponsored_ProductDev        3542
Independent_ProductDev             2117
CompanySponsored_CommunityServ      357
Independent_Heathcare               116
Family/Parent_Preservation           57
CompanySponsored_Heathcare           27
National_Preservation                26
Independent_CommunityServ            26
Regional_Preservation                12
Family/Parent_ProductDev              6
National_ProductDev                   4
Independent_Other                     3
National_Heathcare                    3
Other_Preservation                    3
Other_ProductDev                      1
Regional_ProductDev                   1
Family/Parent_CommunityServ           1
Name: count, dtype: int64

In [61]:
# Limit columns to 10 unique values
limit_unique(
    application_df,
    max_value = 10,
    columns_to_limit = ['AFFILIATION_ORGANIZATION', 'AFFILIATION_USECASE']
)

AFFILIATION_ORGANIZATION
Independent_Trust                16212
CompanySponsored_Association      8479
CompanySponsored_Trust            7206
Independent_Association           1764
Independent_Co-operative           465
Family/Parent_Trust                 55
Independent_Corporation             39
Other                               32
National_Trust                      29
CompanySponsored_Co-operative       18
Name: count, dtype: int64
Number of unique values: 10

AFFILIATION_USECASE
Independent_Preservation          16218
CompanySponsored_Preservation     11779
CompanySponsored_ProductDev        3542
Independent_ProductDev             2117
CompanySponsored_CommunityServ      357
Independent_Heathcare               116
Family/Parent_Preservation           57
Other                                34
CompanySponsored_Heathcare           27
Independent_CommunityServ            26
National_Preservation                26
Name: count, dtype: int64
Number of unique values: 11



### Encoding

In [62]:
# Get all the columns which are categorical
categorical_hot = list(application_df.select_dtypes(include="object").columns)

# Convert categorical data to numeric with `pd.get_dummies`
encoded_columns = pd.get_dummies(application_df[categorical_hot]).astype(int)
encoded_columns.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,...,AFFILIATION_USECASE_CompanySponsored_Heathcare,AFFILIATION_USECASE_CompanySponsored_Preservation,AFFILIATION_USECASE_CompanySponsored_ProductDev,AFFILIATION_USECASE_Family/Parent_Preservation,AFFILIATION_USECASE_Independent_CommunityServ,AFFILIATION_USECASE_Independent_Heathcare,AFFILIATION_USECASE_Independent_Preservation,AFFILIATION_USECASE_Independent_ProductDev,AFFILIATION_USECASE_National_Preservation,AFFILIATION_USECASE_Other
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [63]:
# Split our preprocessed data into our features and target arrays
# Isolate the target array
y = application_df['IS_SUCCESSFUL']

# Isolate the feature array
X = application_df.drop(columns=['IS_SUCCESSFUL'])

# Replace features with encoded equivalents
X.drop(categorical_hot, axis=1, inplace=True)

# Calculate updated feature array
X = pd.concat([X, encoded_columns], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [64]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Hyperparameter Tuning

In [65]:
# Define the model parameters
number_input_features = len(X_train.columns)

# Maximum hidden layers (min. 2 for DL)
max_hidden_layers = 3

# Maximum neurons per hidden layer
max_num_neurons = number_input_features * 2 - 1

# Step count
step_count = 5

# Hidden layer activation functions
activation_functions = [
    'relu', 'leaky_relu', 'tanh',
    'elu', 'selu', 'exponential',
    'softmax', 'softplus'
]

In [66]:
# Initialise the Hyperband tuner
tuner = kt.Hyperband(
    create_model,
    objective = "val_accuracy",
    max_epochs = 20,
    hyperband_iterations = 2
)

Reloading Tuner from ./untitled_project/tuner0.json


In [67]:
# Find the best hyperparameters
tuner.search(
    X_train_scaled,
    y_train,
    epochs = 20,
    validation_data = (X_test_scaled, y_test)
)

## Compile, Train and Evaluate the Best Model

In [68]:
# Get the top 3 model hyperparameters
top3_hyper = tuner.get_best_hyperparameters(3)

for model in top3_hyper:
    pprint(model.values)

{'activation_layer_0': 'softmax',
 'activation_layer_1': 'tanh',
 'activation_layer_2': 'leaky_relu',
 'num_layers': 1,
 'tuner/bracket': 0,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'units_layer_0': 121,
 'units_layer_1': 116,
 'units_layer_2': 106}
{'activation_layer_0': 'selu',
 'activation_layer_1': 'selu',
 'activation_layer_2': 'leaky_relu',
 'num_layers': 1,
 'tuner/bracket': 0,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'units_layer_0': 131,
 'units_layer_1': 71,
 'units_layer_2': 66}
{'activation_layer_0': 'elu',
 'activation_layer_1': 'leaky_relu',
 'activation_layer_2': 'selu',
 'num_layers': 2,
 'tuner/bracket': 2,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'units_layer_0': 101,
 'units_layer_1': 11,
 'units_layer_2': 106}


In [70]:
# Get the top model
best_hyper = top3_hyper[0]
best_hyper.values

{'activation_layer_0': 'softmax',
 'units_layer_0': 121,
 'num_layers': 1,
 'units_layer_1': 116,
 'activation_layer_1': 'tanh',
 'units_layer_2': 106,
 'activation_layer_2': 'leaky_relu',
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [71]:
# Define the model parameters
number_input_features = len(X_train.columns)
hidden_layer0_neurons = best_hyper.values['units_layer_0']
hidden_layer0_activation = best_hyper.values['activation_layer_0']

# Return the number of hidden layers
total_hidden = best_hyper.values['num_layers'] + 1

In [72]:
# Initialise the sequential model
nn = tf.keras.models.Sequential()

# Create the input layer and first hidden layer
nn.add(tf.keras.layers.Dense(
    units = hidden_layer0_neurons,
    activation = hidden_layer0_activation,
    input_dim = number_input_features
))
        
# Create additional hidden layers
for layer in range(1, total_hidden):
    nn.add(tf.keras.layers.Dense(
        units = best_hyper.values[f'units_layer_{layer}'],
        activation = best_hyper.values[f'activation_layer_{layer}']
    ))

# Create the output layer
nn.add(tf.keras.layers.Dense(
    units = 1,
    activation = "sigmoid"
))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 121)               8954      
                                                                 
 dense_5 (Dense)             (None, 116)               14152     
                                                                 
 dense_6 (Dense)             (None, 1)                 117       
                                                                 
Total params: 23223 (90.71 KB)
Trainable params: 23223 (90.71 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [73]:
# Compile the model
nn.compile(
    loss = "binary_crossentropy",
    optimizer = "adam",
    metrics = ["accuracy"]
)

In [74]:
# Train the model
fit_model = nn.fit(
    X_train_scaled,
    y_train,
    epochs = 100,
    verbose = 1
)

Epoch 1/100
804/804 [==============================] - 1s 678us/step - loss: 0.5732 - accuracy: 0.7119
Epoch 2/100
804/804 [==============================] - 1s 693us/step - loss: 0.5510 - accuracy: 0.7294
Epoch 3/100
804/804 [==============================] - 1s 659us/step - loss: 0.5477 - accuracy: 0.7324
Epoch 4/100
804/804 [==============================] - 1s 656us/step - loss: 0.5458 - accuracy: 0.7334
Epoch 5/100
804/804 [==============================] - 1s 683us/step - loss: 0.5439 - accuracy: 0.7320
Epoch 6/100
804/804 [==============================] - 1s 662us/step - loss: 0.5425 - accuracy: 0.7336
Epoch 7/100
804/804 [==============================] - 1s 645us/step - loss: 0.5419 - accuracy: 0.7322
Epoch 8/100
804/804 [==============================] - 1s 667us/step - loss: 0.5411 - accuracy: 0.7346
Epoch 9/100
804/804 [==============================] - 1s 647us/step - loss: 0.5406 - accuracy: 0.7342
Epoch 10/100
804/804 [==============================] - 1s 652us/step - l

In [75]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(
    X_test_scaled,
    y_test,
    verbose = 2
)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5612 - accuracy: 0.7305 - 138ms/epoch - 513us/step
Loss: 0.5612234473228455, Accuracy: 0.7304956316947937


In [76]:
# Print the model architecture
print(f"Input features: {number_input_features}")
print(f"Hidden Layer 0: {hidden_layer0_activation}, {hidden_layer0_neurons}")

for layer in range(1, total_hidden):
    activation = best_hyper.values[f'activation_layer_{layer}']
    neurons = best_hyper.values[f'units_layer_{layer}']
    print(f"Hidden Layer {layer}: {activation}, {neurons}")
    

Input features: 73
Hidden Layer 0: softmax, 121
Hidden Layer 1: tanh, 116


In [24]:
# Export our model to HDF5 file
nn.save('models/AlphabetSoupCharity_iteration5.h5', save_format='h5')

/Users/alyssahondrade/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
